In [97]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
%matplotlib inline

In [98]:
data_COVID_demand = pd.read_csv("data_COVID_demand.csv", sep='\t')

In [99]:
data_COVID_demand = data_COVID_demand.replace(to_replace =",", 
                 value =".", regex=True) 

In [100]:
d10 = data_COVID_demand.loc[data_COVID_demand["category code"] == 10]

In [101]:
d20 = data_COVID_demand.loc[data_COVID_demand["category code"] == 20]

In [102]:
frames = [d10, d20]

data10 = d10
data20 = d20

In [103]:
drivers = ["price", "pctlpt_1_stock", "pctlpt_2_stock", "request_abs", "request_relative", "google_trend_cov","putin_news", "spb_news_1", "spb_news_2", "spb_news_3", "spb_news_4", "spb_news_5", "interfax_china","interfax_tourism", "interfax_shops", "interfax_medicine", "interfax_isolation","item_request"]



In [104]:
from scipy.interpolate import UnivariateSpline

def make_interpolate(data, future, t):
    ynew = data[future].values[-8:][::-1][::2]
    xnew = np.arange(0, t, 1)
    f = UnivariateSpline([0, 4, 8, 11], ynew)
    new_data = f(xnew)
    new_data[new_data < 0] = 0
    return new_data

def make_bad_interpolate(data, future, t):
    y = data[future].values[-6:].astype(float)
    x = np.arange(len(y))
    f = UnivariateSpline(x, y, k=1, s=0, ext=0)
    new_data = np.log(((f(np.arange(len(np.arange(0, t, 1))) + 5)))) / np.log(y[-1]) * y[-1]
    new_data[new_data < 0] = 0
    new_data = np.nan_to_num(new_data, y[-1])
    new_data[new_data > np.max(data[future].values.astype(float))] = np.max(data[future].values.astype(float))
    return new_data
    
data10 = data10.drop(columns = ["category code"])
gb10 = data10.groupby("product_code")
products10 = data10["product_code"].unique()

new_prod_data_10 = []

bad_new_prod_data_10 = []

for prod in products10:
    newDF = pd.DataFrame()
    bad_newDF = pd.DataFrame()
    cur_data = gb10.get_group(prod)
    newDF['product_code'] = [prod] * 12
    bad_newDF['product_code'] = [prod] * 12
    for future in drivers:
        newDF[future] = make_interpolate(cur_data, future, 12)
        bad_newDF[future] = make_bad_interpolate(cur_data, future, 12)
    new_prod_data_10.append(newDF)
    bad_new_prod_data_10.append(bad_newDF)

/Users/mishaantuh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning:

invalid value encountered in log

/Users/mishaantuh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning:

invalid value encountered in less

/Users/mishaantuh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning:

invalid value encountered in true_divide

/Users/mishaantuh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning:

divide by zero encountered in log



In [105]:
data20 = data20.drop(columns = ["category code"])
gb20 = data20.groupby("product_code")
products20 = data20["product_code"].unique()

new_prod_data_20 = []

bad_new_prod_data_20 = []

for prod in products20:
    newDF = pd.DataFrame()
    bad_newDF = pd.DataFrame()
    cur_data = gb20.get_group(prod)
    newDF['product_code'] = [prod] * 12
    bad_newDF['product_code'] = [prod] * 12
    for future in drivers:
        newDF[future] = make_interpolate(cur_data, future, 12)
        bad_newDF[future] = make_bad_interpolate(cur_data, future, 12)
    new_prod_data_20.append(newDF)
    bad_new_prod_data_20.append(bad_newDF)

/Users/mishaantuh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning:

invalid value encountered in log

/Users/mishaantuh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning:

invalid value encountered in less

/Users/mishaantuh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning:

invalid value encountered in true_divide

/Users/mishaantuh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning:

divide by zero encountered in log



In [106]:
cov_drivers = ["week_recovered", "week_deaths", "week_infected", "per_of_recovered", "per_of_deaths", "per_confirmed_from_total",
              "isolation_index"]


In [107]:
cov_data = pd.read_csv("predicts_next_28_week.csv")[cov_drivers][:12]

In [108]:
target = data_COVID_demand["qty"].astype(float).values

In [109]:
X = data_COVID_demand.drop(columns = ["qty", "curr_week", "week_confirmed"]).astype(float)

In [110]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

In [111]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

model = LGBMRegressor(learning_rate=0.1, n_estimators=60).fit(X_train, y_train)
pred = model.predict(X_test)
np.sqrt(mean_squared_error(pred, y_test))

353.8628983429057

In [112]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(learning_rate=0.1, n_estimators=1000).fit(X_train, y_train, silent = True)
pred = model.predict(X_test)
np.sqrt(mean_squared_error(pred, y_test))

317.0100649972537

In [113]:
col = gb20.get_group(prod).drop(columns = ["curr_week", "qty", "week_confirmed", "product_code"]).columns

In [114]:
model = CatBoostRegressor(learning_rate=0.1, n_estimators=1000).fit(X, target, silent = True)

In [115]:
col

Index(['price', 'pctlpt_1_stock', 'pctlpt_2_stock', 'isolation_index',
       'week_recovered', 'week_deaths', 'week_infected', 'per_of_recovered',
       'per_of_deaths', 'per_confirmed_from_total', 'request_abs',
       'request_relative', 'google_trend_cov', 'putin_news', 'spb_news_1',
       'spb_news_2', 'spb_news_3', 'spb_news_4', 'spb_news_5',
       'interfax_china', 'interfax_tourism', 'interfax_shops',
       'interfax_medicine', 'interfax_isolation', 'item_request'],
      dtype='object')

In [165]:
for i in range(len(new_prod_data_10)):
    prod = new_prod_data_10[i]["product_code"][0]
    data_cb = pd.concat([new_prod_data_10[i], cov_data], axis = 1)[col] 
    data_cb_bad = pd.concat([bad_new_prod_data_10[i], cov_data], axis = 1)[col] 
    
    model = CatBoostRegressor(learning_rate=0.1, n_estimators=1000).fit(gb10.get_group(prod)[col], gb10.get_group(prod)["qty"].values, silent = True)
    
    
    new_qty = model.predict(data_cb)
    new_qty[new_qty < 0] = 0
    
    new_qty_bad = model.predict(data_cb_bad)
    new_qty_bad[new_qty_bad < 0] = 0
    
    old_qty = gb10.get_group(prod)["qty"].values
    y1 = old_qty.astype(float)
    y2 = np.concatenate(([old_qty.astype(float)[-1]], new_qty))
    y3 = np.concatenate(([old_qty.astype(float)[-1]], new_qty_bad))
    
    size = len(gb10.get_group(prod)["curr_week"].values) + 11
    date = datetime.datetime.strptime(gb10.get_group(prod)["curr_week"].values[0], '%d-%b-%Y')
    dates_new = []
    dates_new.append(date)
    for i in range(size):
        date += datetime.timedelta(days=7)
        dates_new.append(date)
        
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates_new, y=y1,name = "Real", mode = "lines", line=dict(color='#0D2A63', width=3)))
    fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y2, name = "Good scenario", mode = "lines", line=dict(color='#00A08B', width=3)))
    fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y3,name = "Bad scenario", mode = "lines", line=dict(color='#AF0038', width=3)))
    fig.update_layout(title="category code: 10, product code:" + str(prod),
                       xaxis_title='Days')
    fig.show()

In [168]:
for i in range(len(new_prod_data_20)):
    prod = new_prod_data_20[i]["product_code"][0]
    data_cb = pd.concat([new_prod_data_20[i], cov_data], axis = 1)[col] 
    data_cb_bad = pd.concat([bad_new_prod_data_20[i], cov_data], axis = 1)[col] 
    
    model = CatBoostRegressor(learning_rate=0.1, n_estimators=1000).fit(gb20.get_group(prod)[col], gb20.get_group(prod)["qty"].values, silent = True)
    
    
    new_qty = model.predict(data_cb)
    new_qty[new_qty < 0] = 0
    
    new_qty_bad = model.predict(data_cb_bad)
    new_qty_bad[new_qty_bad < 0] = 0
    
    old_qty = gb20.get_group(prod)["qty"].values
    y1 = old_qty.astype(float)
    y2 = np.concatenate(([old_qty.astype(float)[-1]], new_qty))
    y3 = np.concatenate(([old_qty.astype(float)[-1]], new_qty_bad))
    
    size = len(gb20.get_group(prod)["curr_week"].values) + 11
    date = datetime.datetime.strptime(gb20.get_group(prod)["curr_week"].values[0], '%d-%b-%Y')
    dates_new = []
    dates_new.append(date)
    for i in range(size):
        date += datetime.timedelta(days=7)
        dates_new.append(date)
        
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates_new, y=y1,name = "Real", mode = "lines", line=dict(color='#0D2A63', width=3)))
    fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y2, name = "Good scenario", mode = "lines", line=dict(color='#00A08B', width=3)))
    fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y3,name = "Bad scenario", mode = "lines", line=dict(color='#AF0038', width=3)))
    fig.update_layout(title="category code: 20, product code:" + str(prod),
                       xaxis_title='Days')
    fig.show()

In [118]:
data20["product_code"].unique()[0]

12897

In [161]:
x = np.arange(len(gb20.get_group(319866)["pctlpt_1_stock"].values))
x1 = np.arange(len(new_prod_data_20[-1]["pctlpt_1_stock"].values)) + x[-1]
y1 = gb20.get_group(319866)["pctlpt_1_stock"].values
y2 = new_prod_data_20[-1]["pctlpt_1_stock"].values
y3 = bad_new_prod_data_20[-1]["pctlpt_1_stock"].values

size = len(gb20.get_group(319866)["curr_week"].values) + 11
date = datetime.datetime.strptime(gb20.get_group(319866)["curr_week"].values[0], '%d-%b-%Y')
dates_new = []
dates_new.append(date.strftime("%d %B %Y"))
for i in range(size):
    date += datetime.timedelta(days=7)
    dates_new.append(date)

fig = go.Figure()
fig.add_trace(go.Scatter(x=dates_new, y=y1,name = "Real", mode = "lines", line=dict(color='#0D2A63', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y2, name = "Good scenario", mode = "lines", line=dict(color='#00A08B', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y3,name = "Bad scenario", mode = "lines", line=dict(color='#AF0038', width=3)))
fig.update_layout(title="Доля магазинов в ситуации дефицита товара category code: 20, product code 319866",
                   xaxis_title='Days')
    

In [158]:
x = np.arange(len(gb20.get_group(144400)["pctlpt_1_stock"].values))
x1 = np.arange(len(new_prod_data_20[-1]["pctlpt_1_stock"].values)) + x[-1]
y1 = gb20.get_group(144400)["pctlpt_1_stock"].values
y2 = new_prod_data_20[-4]["pctlpt_1_stock"].values
y3 = bad_new_prod_data_20[-4]["pctlpt_1_stock"].values

size = len(gb20.get_group(144400)["curr_week"].values) + 11
date = datetime.datetime.strptime(gb20.get_group(144400)["curr_week"].values[0], '%d-%b-%Y')
dates_new = []
dates_new.append(date.strftime("%d %B %Y"))
for i in range(size):
    date += datetime.timedelta(days=7)
    dates_new.append(date)

fig = go.Figure()
fig.add_trace(go.Scatter(x=dates_new, y=y1,name = "Real", mode = "lines", line=dict(color='#0D2A63', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y2, name = "Good scenario", mode = "lines", line=dict(color='#00A08B', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y3,name = "Bad scenario", mode = "lines", line=dict(color='#AF0038', width=3)))
fig.update_layout(title="Доля магазинов в ситуации дефицита товара category code: 20, product code 144400",
                   xaxis_title='Days')
    

In [156]:
x = np.arange(len(gb20.get_group(125249)["pctlpt_1_stock"].values))
x1 = np.arange(len(new_prod_data_20[2]["pctlpt_1_stock"].values)) + x[-1]
y1 = gb20.get_group(125249)["pctlpt_1_stock"].values
y2 = new_prod_data_20[2]["pctlpt_1_stock"].values
y3 = bad_new_prod_data_20[2]["pctlpt_1_stock"].values


size = len(gb20.get_group(125249)["curr_week"].values) + 11
date = datetime.datetime.strptime(gb20.get_group(125249)["curr_week"].values[0], '%d-%b-%Y')
dates_new = []
dates_new.append(date.strftime("%d %B %Y"))
for i in range(size):
    date += datetime.timedelta(days=7)
    dates_new.append(date)
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates_new, y=y1,name = "Real", mode = "lines", line=dict(color='#0D2A63', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y2, name = "Good scenario", mode = "lines", line=dict(color='#00A08B', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y3,name = "Bad scenario", mode = "lines", line=dict(color='#AF0038', width=3)))
fig.update_layout(title="Доля магазинов в ситуации дефицита товара category code: 20, product code 125249",
                   xaxis_title='Days')
    

In [150]:
x = np.arange(len(gb10.get_group(307142)["price"].values))
x1 = np.arange(len(new_prod_data_10[2]["price"].values)) + x[-1]
y1 = gb10.get_group(307142)["price"].values.astype(float)
y2 = new_prod_data_10[-1]["price"].values
y3 = bad_new_prod_data_10[-1]["price"].values

size = len(gb10.get_group(307142)["curr_week"].values) + 11
date = datetime.datetime.strptime(gb10.get_group(307142)["curr_week"].values[0], '%d-%b-%Y')
dates_new = []
dates_new.append(date.strftime("%d %B %Y"))
for i in range(size):
    date += datetime.timedelta(days=7)
    dates_new.append(date)

fig = go.Figure()
fig.add_trace(go.Scatter(x=dates_new, y=y1,name = "Real", mode = "lines", line=dict(color='#0D2A63', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y2, name = "Good scenario", mode = "lines", line=dict(color='#00A08B', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y3,name = "Bad scenario", mode = "lines", line=dict(color='#AF0038', width=3)))
fig.update_layout(title="Цена товара category code: 10, product code 307142",
                   xaxis_title='Days')
    

In [149]:
x = np.arange(len(gb10.get_group(27462)["price"].values))
x1 = np.arange(len(new_prod_data_10[2]["price"].values)) + x[-1]
y1 = gb10.get_group(27462)["price"].values.astype(float)
y2 = new_prod_data_10[4]["price"].values
y3 = bad_new_prod_data_10[4]["price"].values

size = len(gb10.get_group(27462)["curr_week"].values) + 11
date = datetime.datetime.strptime(gb10.get_group(27462)["curr_week"].values[0], '%d-%b-%Y')
dates_new = []
dates_new.append(date.strftime("%d %B %Y"))
for i in range(size):
    date += datetime.timedelta(days=7)
    dates_new.append(date)

    
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates_new, y=y1,name = "Real", mode = "lines", line=dict(color='#0D2A63', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y2, name = "Good scenario", mode = "lines", line=dict(color='#00A08B', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y3,name = "Bad scenario", mode = "lines", line=dict(color='#AF0038', width=3)))
fig.update_layout(title="Цена товара category code: 10, product code 27462",
                   xaxis_title='Days')

In [145]:
x = np.arange(len(gb10.get_group(207140)["price"].values))
x1 = np.arange(len(new_prod_data_10[20]["price"].values)) + x[-1]
y1 = gb10.get_group(207140)["price"].values.astype(float)
y2 = new_prod_data_10[20]["price"].values
y3 = bad_new_prod_data_10[20]["price"].values


size = len(gb10.get_group(27462)["curr_week"].values) + 11
date = datetime.datetime.strptime(gb10.get_group(207140)["curr_week"].values[0], '%d-%b-%Y')
dates_new = []
dates_new.append(date.strftime("%d %B %Y"))
for i in range(size):
    date += datetime.timedelta(days=7)
    dates_new.append(date)

    
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates_new, y=y1,name = "Real", mode = "lines", line=dict(color='#0D2A63', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y2, name = "Good scenario", mode = "lines", line=dict(color='#00A08B', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y3,name = "Bad scenario", mode = "lines", line=dict(color='#AF0038', width=3)))
fig.update_layout(title="Цена товара category code: 10, product code 207140",
                   xaxis_title='Days')

In [144]:
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()
fig.add_trace(go.Scatter(x=dates_new, y=y1,name = "Real", mode = "lines", line=dict(color='#0D2A63', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y2, name = "Good scenario", mode = "lines", line=dict(color='#00A08B', width=3)))
fig.add_trace(go.Scatter(x=dates_new[len(y1) - 1:], y=y3,name = "Bad scenario", mode = "lines", line=dict(color='#AF0038', width=3)))
fig.update_layout(title="Цена товара category code: 10, product code 207140",
                   xaxis_title='Days')